## SILVER TO GOLD LAYER


In [34]:
import pandas as pd
from global_paths import GlobalPaths
from common_utilities import TradeHistory

### TradeHistory Paths


In [35]:
# Initialize TradeHistory Paths
TradeHistoryPath = GlobalPaths(source_name="DATA", object_name="TradeHistory")
TradeHistorySilverPath = TradeHistoryPath.createLayer(layer_name="SILVER")
TradeHistorySilverFile = TradeHistorySilverPath.joinpath("TradeHistory_data.csv")
TradeHistoryGoldPath = TradeHistoryPath.createLayer(layer_name="GOLD")
TradeHistoryGoldFile = TradeHistoryGoldPath.joinpath("TradeHistory_data.csv")

# Initialize Symbol Paths
SymbolPath = GlobalPaths(source_name="DATA", object_name="Symbol")
SymbolSilverPath = SymbolPath.createLayer(layer_name="SILVER")
SymbolSilverFile = SymbolSilverPath.joinpath("Symbol_data.csv")

### Gold Layer - Trade History


In [36]:
# read the csv file
df_TradeHistory = pd.read_csv(TradeHistorySilverFile)

# replace scrip code to compnay name
df_Symbol = pd.read_csv(SymbolSilverFile)
df_Symbol["scrip_code"] = df_Symbol["scrip_code"].astype(str)

# Convert 'datetime' to datetime type
df_TradeHistory["datetime"] = pd.to_datetime(df_TradeHistory["datetime"])

# Merge df_TradeHistory with df_Symbol on the matching columns
df_TradeHistory = df_TradeHistory.merge(
    df_Symbol[["scrip_code", "security_id"]],
    left_on="scrip_code",
    right_on="scrip_code",
    how="left",
)


# Assign the new column 'stock_name' in df_TradeHistory to the values from 'security_id'
df_TradeHistory["stock_name"] = df_TradeHistory["security_id"].combine_first(
    df_TradeHistory["company"]
)


# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by="datetime")

data_dict = list()
trade_history = dict()
for _, row in df_TradeHistory.iterrows():
    stock_name = row["stock_name"]
    if stock_name not in trade_history:
        trade_history[stock_name] = TradeHistory(stock_name)

    if str(row["trade_type"]).upper() == "BUY":
        row["buy_price"] = row["trade_price"]
        trade_history[stock_name].trade_price.append(row["trade_price"])
        trade_history[stock_name].trade_quantity.append(row["trade_quantity"])
    elif str(row["trade_type"]).upper() == "SELL":
        row["sell_price"] = row["trade_price"]
        row["buy_price"] = trade_history[stock_name].fifo_sell_calc(
            row["trade_quantity"]
        )
    else:
        raise Exception(f'{row["trade_type"]} was never excepected')

    row["holding_quantity"] = trade_history[stock_name].holding_quantity()
    row["avg_price"] = trade_history[stock_name].calc_avg_price()
    data_dict.append(row)

df_TradeHistory = pd.DataFrame(data_dict)
df_TradeHistory = df_TradeHistory.round(2)
df_TradeHistory = df_TradeHistory[
    [
        "datetime",
        "segment",
        "stock_name",
        "trade_type",
        "trade_quantity",
        "buy_price",
        "sell_price",
        "holding_quantity",
        "avg_price",
    ]
]


# Save the result as a csv file
df_TradeHistory.to_csv(TradeHistoryGoldFile, index=None)
df_TradeHistory.info()
print("Gold Layer csv file for trade history successfully created at:")
print(TradeHistoryGoldFile.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 139 entries, 0 to 138
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   datetime          139 non-null    datetime64[ns]
 1   segment           139 non-null    object        
 2   stock_name        139 non-null    object        
 3   trade_type        139 non-null    object        
 4   trade_quantity    139 non-null    float64       
 5   buy_price         139 non-null    float64       
 6   sell_price        67 non-null     float64       
 7   holding_quantity  139 non-null    float64       
 8   avg_price         139 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 10.9+ KB
Gold Layer csv file for trade history successfully created at:
C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\TradeHistory\TradeHistory_data.csv
